## Q1. 가장 먼저 학습 데이터를 준비해보도록 하겠습니다. MNIST 데이터셋을 직접 Load해 봅시다. 데이터셋을 로드하고 DataLoader를 구현해보세요.



### DataLoader를 이용해 MNIST 데이터셋을 로드해봅시다.

In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

#data loader를 직접 구현해보자.
train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle=True)

## Q2.데이터가 준비가 되었다면, 이제 그 데이터를 학습할 모델을 구현할 차례입니다. 그 후 모델 안의 가중치를 초기화시켜보세요. 입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.

### MNIST 입력의 크기는 28x28입니다.

### 여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰보세요.

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias=True).to(device)
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 0.5597, -2.2007,  0.3742,  ...,  1.1078,  0.7588, -0.8797],
        [ 1.9275, -0.7699, -0.9300,  ..., -0.7964, -0.7338, -1.2891],
        [ 0.2891,  0.2262, -0.4337,  ..., -0.8227, -0.3446,  1.2654],
        ...,
        [ 0.9286,  0.4095,  1.0091,  ...,  2.6134, -0.7179,  1.9400],
        [-1.3079, -0.4623, -0.7608,  ..., -2.1933,  0.1715,  0.9602],
        [-1.7361,  1.7099,  0.9369,  ...,  0.7345, -0.6297, -0.3486]],
       requires_grad=True)

## Q3. 위에서 구현한 모델을 학습시키기 위해서는 loss 함수와 opitmizer가 필요합니다. 아래 제시된 loss 함수와 optimizer를 구현해보세요. Loss 함수와 optimizer는 모델 안의 가중치를 업데이트 할 때 사용됩니다.

### 옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [5]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

## Q4. 3번 문제까지 해결하셨다면, 이제 학습을 위한 준비는 거의 끝났다고 볼 수 있습니다. 위 구현 함수들을 이용해 학습 Loop를 구현해보세요.

### 위에서 구현한 모델, optimzer, loss fn 등을 이용해 학습을 구현해주세요.

In [14]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)
        
        outputs = linear(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}, Accuracy: {:.2f}%'.format( epoch+1, training_epochs,
            i+1, len(train_loader), loss.item(), accuracy.item()* 100))

Epoch [1/15], Step [100/600], Loss:1.7296, Accuracy: 72.00%
Epoch [1/15], Step [200/600], Loss:0.9654, Accuracy: 78.00%
Epoch [1/15], Step [300/600], Loss:0.8391, Accuracy: 83.00%
Epoch [1/15], Step [400/600], Loss:1.2242, Accuracy: 75.00%
Epoch [1/15], Step [500/600], Loss:1.2674, Accuracy: 79.00%
Epoch [1/15], Step [600/600], Loss:0.3690, Accuracy: 93.00%
Epoch [2/15], Step [100/600], Loss:1.0288, Accuracy: 78.00%
Epoch [2/15], Step [200/600], Loss:0.6694, Accuracy: 84.00%
Epoch [2/15], Step [300/600], Loss:0.7971, Accuracy: 82.00%
Epoch [2/15], Step [400/600], Loss:0.7487, Accuracy: 87.00%
Epoch [2/15], Step [500/600], Loss:0.8489, Accuracy: 86.00%
Epoch [2/15], Step [600/600], Loss:0.4941, Accuracy: 91.00%
Epoch [3/15], Step [100/600], Loss:1.2166, Accuracy: 79.00%
Epoch [3/15], Step [200/600], Loss:0.6136, Accuracy: 86.00%
Epoch [3/15], Step [300/600], Loss:1.2625, Accuracy: 76.00%
Epoch [3/15], Step [400/600], Loss:0.5604, Accuracy: 87.00%
Epoch [3/15], Step [500/600], Loss:0.778

## Q5. 학습이 완료되면, 모델이 잘 동작하는지 테스트가 필요합니다. 데이터로드 파트에서 준비했던 테스트 데이터를 이용해 테스트를 진행해봅시다. 아래 테스트 코드를 완성해보세요.

In [13]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)
        outputs = linear(imgs)
        _, argmax = torch.max(outputs, 1) #max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()
        
        print('Test accuracy for {} images: {:.2f}%'.format(total, correct/total*100))

Test accuracy for 100 images: 63.00%
Test accuracy for 200 images: 63.00%
Test accuracy for 300 images: 62.33%
Test accuracy for 400 images: 63.00%
Test accuracy for 500 images: 63.80%
Test accuracy for 600 images: 61.83%
Test accuracy for 700 images: 62.14%
Test accuracy for 800 images: 63.00%
Test accuracy for 900 images: 63.22%
Test accuracy for 1000 images: 63.30%
Test accuracy for 1100 images: 64.36%
Test accuracy for 1200 images: 64.67%
Test accuracy for 1300 images: 65.31%
Test accuracy for 1400 images: 65.43%
Test accuracy for 1500 images: 65.67%
Test accuracy for 1600 images: 65.56%
Test accuracy for 1700 images: 65.53%
Test accuracy for 1800 images: 65.17%
Test accuracy for 1900 images: 65.47%
Test accuracy for 2000 images: 65.70%
Test accuracy for 2100 images: 66.00%
Test accuracy for 2200 images: 66.18%
Test accuracy for 2300 images: 65.83%
Test accuracy for 2400 images: 66.12%
Test accuracy for 2500 images: 66.16%
Test accuracy for 2600 images: 66.46%
Test accuracy for 270